This notebook is part of the tutorials materials to Spatial Mapping of b-vlaue of earthquakes

### Spatial Mapping of b-value of earthquakes at Salton Trough and Imperial Valley, Southern California ###

The earthquake size distribution can be described by a power-law with an exponent \( b \), which shows the Gutenberg-Richter relationship (Gutenberg & Richter, 1944):

$$ \log(N) = a - bM $$

Where $b$ is the slope of the number of events $N$ with greater than magnitude $M$ provides the seismic activity and $a$ provides productivity. The $b$-value will be computed using the maximum-likelihood estimator (Aki, 1965; Bender, 1983; Utsu, Ogata, S, & Matsu’ura, 1995):
$$\ b = \frac{1}{{\bar{M} - M_c}} \log_{10}(e) \quad$$
Where $\bar{M}$ is the mean magnitude, $e=\exp(1)$, and $M_c$ is the magnitude of completeness adjusted for bin size to account for potential bin size biases (Guo & Ogata, 1997; Utsu et al., 1995). We will estimate the magnitude of completeness using the maximum curvature method. The statistical uncertainty for maximum likelihood b-value estimates can be determined from (Shi & Bolt, 1982).
$$ \sigma _b = \frac{b^2}{\log(e)} \sqrt{\frac{\sum_{i=1}^{n} (M_i - \bar{M})}{n(n-1)}} \quad $$
Where n is the number of events above $M_c$, $M_i$ is the magnitude of individual AE events and $\bar{M}$ is the mean magnitude for events greater or equal than $M_c$.

The b-value in Gutenberg-Richter relation is a crucial parameter in earthquake seismology that is obtained from the Gutenberg-Richter frequency-magnitude distribution. The b-value is the defined by the slope of magnitude frequency distribution of earthquakes and is independent of whether global, regional or local seismicity in analyzed (Fiedler, Hainzl, Z ̈oller, & Holschneider, 2018-10-01).

### Instructions for Defining the Region of Interest for Earthquake Analysis ###
To define the region of interest for example, the Salton Trough/Imperial Valley area, you need to specify the following parameters:

#### Geographic Boundaries: ####
- Minimum Latitude = 32.00 
- Maximum Latitude = 36.00
- Minimum Longitude = -120.0
- Maximum Longitude = -111.0
#### Earthquake Characteristics: ####
- Minimum Magnitude = -1.0
- Maximum Magnitude = 9.0
- Minimum Depth  = -5.0
- Maximum Depth  = 30.0
#### Time Boundaries: ####
- Start Time = 1977:01:01:00
- End Time   = 2024:06:07:00 (Today)

use above information when accessing earthquake catalog data. you can excess catalog data from Southern California Earthquake Data Center (SCEDC) from [here](https://service.scedc.caltech.edu/eq-catalogs/date_mag_loc.php) After accessing data you can save under the data directory. 


### 1. Import required  and packages ###

In [15]:
import os
from io import StringIO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pygmt #for plotting purposes

### 2. Managae input output directories ###

In [10]:
path = os.getcwd()
dataDir  = os.path.join(path, "data",)
plotDir= os.path.join(path, "plots")
if not os.path.exists(plotDir):
    os.mkdirs(plotDir)
    print(f"Created Output Directory: {plotDir}")
else:
    print("Output Directory already exists.")

Output Directory already exists.


### Alterntive way to download earthquakes catalog data ###

In [8]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import csv

# Initialize the client (for the Southern California Earthquake Data Center (SCEDC))
client = Client("SCEDC")

# Define the output CSV file name
output_csv = "data/event_catalog.csv"

# Define the parameters for the catalog search
start_time = UTCDateTime("2022-01-01")
end_time = UTCDateTime("2022-12-31")
min_magnitude = 3.0
max_magnitude = 6.0
min_latitude = 32.0
max_latitude = 36.0
min_longitude = -120.0
max_longitude = -114.0
min_depth = 0.0
max_depth = 30.0

try:
    # Fetch the catalog data using the specified parameters
    catalog = client.get_events(starttime=start_time, endtime=end_time,
                                minmagnitude=min_magnitude, maxmagnitude=max_magnitude,
                                minlatitude=min_latitude, maxlatitude=max_latitude,
                                minlongitude=min_longitude, maxlongitude=max_longitude,
                                mindepth=min_depth, maxdepth=max_depth)
    
    # Print the number of events in the catalog
    print(f"Number of events in the catalog: {len(catalog)}")
    

    # Write the event details to the CSV file
    with open(output_csv, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        # Write the header
        csv_writer.writerow(["EventID", "OriginTime", "Magnitude", "Latitude", "Longitude", "Depth"])
        
        # Iterate over the events and write the details for each event
        for event in catalog:
            # Extract the event ID by splitting the URI
            event_id = event.resource_id.id.split('=')[-1]
            origin = event.origins[0]
            magnitude = event.magnitudes[0].mag
            csv_writer.writerow([event_id, origin.time, magnitude, origin.latitude, origin.longitude, origin.depth / 1000])
    
    print(f"Seismic event catalog has been saved to {output_csv}")

except Exception as e:
    print(f"An error occurred: {e}")


Number of events in the catalog: 92
Seismic event catalog has been saved to data/seismic_event_catalog.csv


### 3. Load the event catalog downloaded from step 1 ###

In [17]:
catalogFilename = os.path.join(path, "data","testCat.txt")

In [25]:
headerName = [ "date", "time", "ET", "GT", "magnitude", "M", "latitude", "longitude", "depth", "Q", "EVID", "NPH", "NGRM",]
catalog_df = pd.read_csv(
    catalogFilename,
    delim_whitespace=True,
    header=0,
    names=headerName,
    comment="#",
    index_col=False,
    skiprows=2,
)

catalog_df

,date,time,ET,GT,magnitude,M,latitude,longitude,depth,Q,EVID,NPH,NGRM
0,2023/06/06,01:55:57.65,eq,l,1.47,l,33.98700,-116.99967,15.3,A,40479184.0,91.0,2696.0
1,2023/06/06,04:32:58.60,eq,l,1.70,l,33.97933,-116.82100,19.1,A,40479216.0,125.0,2893.0
2,2023/06/06,05:05:19.29,eq,l,1.26,l,35.86450,-117.68750,11.1,A,40479224.0,30.0,1725.0
3,2023/06/06,05:51:47.34,eq,l,0.40,l,33.66950,-116.76650,16.4,A,40479240.0,28.0,1513.0
4,2023/06/06,06:41:09.62,eq,l,0.54,l,33.45217,-116.57783,11.2,A,40479264.0,38.0,1458.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14074,2024/06/05,22:34:07.01,eq,l,1.16,l,35.95583,-117.92683,6.1,A,40615559.0,49.0,648.0
14075,2024/06/05,23:15:48.49,eq,l,0.83,l,35.88767,-117.70300,2.9,A,40615591.0,38.0,412.0
14076,2024/06/05,23:22:30.20,eq,l,1.92,l,34.56750,-116.46233,2.0,A,40615599.0,56.0,2930.0
14077,2024/06/05,23:25:25.38,eq,l,1.38,l,35.03433,-117.67883,-0.8,A,40615607.0,45.0,2063.0


In [3]:
import pandas as pd
import requests
from io import StringIO

# Send request to GitHub API to get list of files in directory
url = "https://api.github.com/repos/SCEDC/SCEDC-catalogs/contents/SCSN"
response = requests.get(url)
files = response.json()

# Filter files to only include those with "year.catalog" naming convention
catalog_files = [file["name"] for file in files if "catalog" in file["name"]]

# Read each catalog file into a pandas dataframe
catalog_dfs = {}
for file in catalog_files:
    url = f"https://raw.githubusercontent.com/SCEDC/SCEDC-catalogs/master/SCSN/{file}"
    response = requests.get(url)
    catalog_df = pd.read_csv(StringIO(response.text), delimiter="|")
    catalog_dfs[file] = catalog_df

print(catalog_dfs)

KeyboardInterrupt: 

In [4]:
# def read_catalog_files(start_year, end_year):
#     catalog_dfs = {}
#     for year in range(start_year, end_year+1):
#         file_name = f"{year}.catalog"
#         url = f"https://raw.githubusercontent.com/SCEDC/SCEDC-catalogs/master/SCSN/{file_name}"
#         response = requests.get(url)
#         catalog_df = pd.read_csv(StringIO(response.text), delimiter="|")
#         catalog_dfs[file_name] = catalog_df
#     return catalog_dfs
import pandas as pd
import requests
from io import StringIO
def read_catalog_files(start_year, end_year):
    catalog_dfs = {}
    for year in range(start_year, end_year+1):
        file_name = f"{year}.catalog"
        url = f"https://raw.githubusercontent.com/SCEDC/SCEDC-catalogs/master/SCSN/{file_name}"
        response = requests.get(url)
        # Skip the header lines
        lines = response.text.splitlines()[6:]
        # Join the lines into a single string
        data = "\n".join(lines)
        # Create a StringIO object to read the data
        data_io = StringIO(data)
        # Read the data into a pandas dataframe
        catalog_df = pd.read_csv(data_io, delim_whitespace=True, header=None, names=[
            "YEAR", "MONTH", "DAY", "HOUR", "MINUTE", "SECOND", "LATITUDE", "LONGITUDE", "Q", "MAG", "DEPTH", "NPH", "RMS", "EVID"
        ])
        catalog_dfs[file_name] = catalog_df
    return catalog_dfs

start_year = 2022
end_year = 2023
catalog_dfs = read_catalog_files(start_year, end_year)
print(catalog_dfs)

{'2022.catalog':                                                            YEAR    MONTH  \
#                                                  and     -114  degrees   
                                                   search  page       at   
###############################################... NaN      NaN      NaN   
#YYY                                               MM        DD       HH   
2022                                               01        01       00   
...                                                         ...      ...   
                                                   12        31       22   
                                                   12        31       23   
                                                   12        31       23   
                                                   12        31       23   
###                                                Number    of     rows   

                                                                      

In [ ]:
!